In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# Change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter_Project1 import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Password1!"

# Connect to database via CRUD Module
db = AnimalShelter()

# Class read method must support return of list object and accept projection JSON input
# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the DataFrame here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new DataFrame that does not contain the dropped column(s)
if not df.empty and '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#rescue_types = ['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking', 'Reset']

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Add logo image with unique identifier
app.layout = html.Div([
    # Add logo image
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '100px'}),
        html.H3('Dashboard by David King')  # Unique identifier
    ]),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'ALL'},
                {'label': 'Dogs', 'value': 'Dog'},
                {'label': 'Cats', 'value': 'Cat'},
                {'label': 'Other', 'value': 'Other'}
            ],
            value='ALL'  # Default value
        ),
        dcc.RadioItems(
            id='additional-filter',
            options=[
                {'label': 'All', 'value': 'ALL'},
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
            ],
            value='ALL'  # Default value
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        row_selectable='single',  # Allow single row selection
        selected_rows=[]  # Start with no rows selected
        # FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        # If you completed the Module Six Assignment, you can copy in the code you created here 
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the data table based on filter selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'),
     Input('additional-filter', 'value')]
)
def update_dashboard(filter_type, additional_filter):
    dff = df

    # Apply dropdown filter
    if additional_filter == 'ALL':
        if filter_type == 'ALL':
            dff = dff[dff['animal_type'].isin(['Cat','Dog','Other'])]
        elif filter_type == 'Cat':
            dff = dff[dff['animal_type'].isin(['Cat'])]
        elif filter_type == 'Dog':
            dff = dff[dff['animal_type'].isin(['Dog'])]
        elif filter_type == 'Other':
            dff = dff[dff['animal_type'].isin(['Other'])]

    # Apply radio button filter
    # Apply radio button filter only for dogs
    if additional_filter != 'ALL':
        if filter_type != 'Dog' and filter_type != 'ALL':
            # If filter_type is not 'Dog', clear data
            dff = pd.DataFrame(columns=df.columns)  # Empty DataFrame
        else:
            # Filter for dogs with the specified additional filter
            if additional_filter == 'Water Rescue':
                dff = dff[dff['animal_type'] == 'Dog']
                dff = dff[dff.breed.isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])
                        & (dff.sex_upon_outcome == 'Intact Female')
                        & ((dff.age_upon_outcome_in_weeks >= 26) & (dff.age_upon_outcome_in_weeks <= 156))]
            elif additional_filter == 'Mountain or Wilderness Rescue':
                dff = dff[dff['animal_type'] == 'Dog']
                dff = dff[dff.breed.isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])
                        & (dff.sex_upon_outcome == 'Intact Male')
                        & ((dff.age_upon_outcome_in_weeks >= 26) & (dff.age_upon_outcome_in_weeks <= 156))]
            elif additional_filter == 'Disaster or Individual Tracking':
                dff = dff[dff['animal_type'] == 'Dog']
                dff = dff[dff.breed.isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])
                        & (dff.sex_upon_outcome == 'Intact Male')
                        & ((dff.age_upon_outcome_in_weeks >= 20) & (dff.age_upon_outcome_in_weeks <= 300))]

    return dff.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty or 'breed' not in dff.columns:
        return [
            dcc.Graph(
                figure=px.pie(
                    pd.DataFrame(columns=['breed']), 
                    names='breed', 
                    title='Preferred Animals'
                )
            )
        ]
    
    # Generate the pie chart
    fig = px.pie(dff, names='breed', title='Preferred Animals')

    return [
        dcc.Graph(figure=fig)
    ]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        selected_columns = []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
# Update the map callback function
# Map callback that displays markers for the locations of animals from them viewport data
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'selected_rows')],
    [State('datatable-id', 'data')]
)
def update_map(selected_rows, data):
    # Convert data to DataFrame
    df = pd.DataFrame(data)
    
    if not selected_rows:
        # Return an empty map or a default map if no row is selected
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id")
            ])
        ]

    # Get the selected row's index
    selected_index = selected_rows[0]
    selected_row = df.iloc[selected_index]

    # Return the map with the selected animal
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[selected_row['location_lat'], selected_row['location_long']],
                children=[
                    dl.Tooltip(selected_row['breed']),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(selected_row['name']),
                        html.H3("Animal ID"),
                        html.P(selected_row['animal_id'])
                    ])
                ]
            )
        ])
    ]
app.run_server(debug=True)

ModuleNotFoundError: No module named 'dash_leaflet'